In [4]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('csv/metadata_text_merged_cleaned.csv')

In [27]:
nationalities = ['dutch', 'french', 'flemish', 'portuguese', 'danish', 'brandenberg', 'english']

counter = 0

for nationality in nationalities:
    for text in df['text_cleaned_dealiased']:
        if re.search(nationality + '.{0,15}interloper', text) is not None:
            counter += 1
    
    print '{}:'.format(nationality), counter
    counter = 0

dutch: 90
french: 5
flemish: 3
portuguese: 3
danish: 0
brandenberg: 0
english: 35
